In [27]:
import TIVlib as tiv
import numpy as np

In [28]:
from essentia.standard import MonoLoader, Windowing, Spectrum, SpectralPeaks, FrameGenerator, HPCP

def file_to_hpcp(filename):
  audio = MonoLoader(filename=filename)()
  windowing = Windowing(type='blackmanharris62')
  spectrum = Spectrum()
  spectral_peaks = SpectralPeaks(orderBy='magnitude',
                                    magnitudeThreshold=0.001,
                                    maxPeaks=20,
                                    minFrequency=20,
                                    maxFrequency=8000)
  hpcp = HPCP(maxFrequency=8000,
              normalized='unitSum')

  spec_group = []
  hpcp_group = []


  for frame in FrameGenerator(audio,frameSize=1024,hopSize=512):
    windowed = windowing(frame)
    fft = spectrum(windowed)
    frequencies, magnitudes = spectral_peaks(fft)
    final_hpcp = hpcp(frequencies, magnitudes)

    spec_group.append(fft)
    hpcp_group.append(final_hpcp)
    
  mean_hpcp = np.mean(np.array(hpcp_group).T, axis = 1)
  return mean_hpcp


In [29]:
import librosa

def file_to_chroma(filename):
    audio = librosa.core.load(filename)
    chromagram = librosa.feature.chroma_stft(audio)
    print (chromagram)
    mean_chromagram = np.mean(chromagram,axis = 0)
    return mean_chromagram
    

In [30]:
metal = "./audio_files/metal.wav"
glass = "./audio_files/glass.wav"

metal_hpcp = file_to_hpcp(metal)
glass_hpcp = file_to_hpcp(glass)
#metal_chroma = file_to_chroma(metal)
#glass_chroma = file_to_chroma(glass)

print(metal_hpcp)
print(glass_hpcp)
#print(metal_chroma)
#print(glass_chroma)


[0.01680101 0.01739293 0.02135463 0.03086618 0.03273978 0.04162527
 0.02266426 0.05893418 0.61454886 0.02947111 0.02630138 0.02236531]
[0.0000000e+00 7.1136880e-04 7.5755358e-02 3.0303048e-03 0.0000000e+00
 5.3012510e-08 7.4694995e-05 0.0000000e+00 8.4052950e-02 0.0000000e+00
 2.7699830e-06 8.8697798e-06]


In [31]:
metal_tiv = tiv.tiv.TIV.from_pcp(metal_hpcp)

ValueError: operands could not be broadcast together with shapes (15,) (6,) 

In [ ]:
tiv.tiv.key()